# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [8]:
from azureml.core import Dataset
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform, randint
from azureml.core import Environment, ScriptRunConfig
import joblib
import os
from azureml.data.dataset_factory import TabularDatasetFactory

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive_experiment'

experiment=Experiment(ws, experiment_name)

In [3]:
#Source of the data: https://raw.githubusercontent.com/aiplanethub/Datasets/master/HR_comma_sep.csv

# We first check if the data is already loaded inthe Workspace. Otherwise, create it from the file

found = False
key = "HR Dataset"
description_text = "HR Dataset for capstone project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/aiplanethub/Datasets/master/HR_comma_sep.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000
mean,0.612834,0.716102,3.803054,201.050337,3.498233,0.144610,0.238083,0.021268
std,0.248631,0.171169,1.232592,49.943099,1.460136,0.351719,0.425924,0.144281
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


## Cluster creation

In [4]:
cluster_name = "cluster-udacity"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster in the provided workspace, so we use it.')
except ComputeTargetException:
    comp_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=3)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, comp_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster in the provided workspace, so we use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [93]:
!cat train.py

from sklearn.linear_model import LogisticRegression
import argparse
import os
import numpy as np
from sklearn.metrics import (accuracy_score,
                             f1_score,
                             roc_auc_score,
                             roc_curve,
                             confusion_matrix,
                             classification_report)
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset, Datastore


def clean_data(datos):
    
    data = datos.to_pandas_dataframe().dropna()

    # Map salary into integers
    salary_map = {"low": 0, "medium": 1, "high": 2}
    data["salary"] = data["salary"].map(salary_map)

    # Create dummy variables for department feature
    dept = pd.get_dummies(data.Department, prefix="dept")
    data.drop("Department",inplace=T

In [94]:
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
import numpy as np

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, 
                                        evaluation_interval = 2,
                                        delay_evaluation = 10)

#TODO: Create the different params that you will be using during training
max_depth_range = choice(4,5,6,7,8,9,10)
min_samples_leaf_range = choice(2,3,4,5,6,7)

param_sampling = RandomParameterSampling( {
    "--C" : uniform(0.0,10.0),
    "--max_iter" : choice(10,20,30,40,50,60,70,80,90,100)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')


#TODO: Create your estimator and hyperdrive config
#estimator = ScriptRunConfig(
#    source_directory='.',
#    script='train.py',
#    compute_target=cluster_name,
#    environment=sklearn_env
#)

#hyperdrive_run_config = HyperDriveConfig(
#    run_config=estimator,
#    hyperparameter_sampling=param_sampling, 
#    primary_metric_name='AUC',
#    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
#    policy=early_termination_policy,
#    max_total_runs=50,
#    max_concurrent_runs=5
#)
estimator = ScriptRunConfig(source_directory='.',
                            command=['python', 'train.py'],
                            compute_target=cluster_name,
                            environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator, 
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name='Accuracy',
                             primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs = 4,
                             max_concurrent_runs = 2)

In [95]:
#TODO: Submit your experiment

# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = experiment.submit(config=hyperdrive_run_config) 



## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [96]:
#show run details with the widget.
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [97]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_35e23ab9-4f03-48c1-825a-b15a503b4327
Web View: https://ml.azure.com/runs/HD_35e23ab9-4f03-48c1-825a-b15a503b4327?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-242066/workspaces/quick-starts-ws-242066&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-09-30T20:17:35.653329][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space
[2023-09-30T20:17:36.0859440Z][SCHEDULER][INFO]Scheduling job, id='HD_35e23ab9-4f03-48c1-825a-b15a503b4327_0' 
[2023-09-30T20:17:36.125510][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.
[2023-09-30T20:17:36.2354612Z][SCHEDULER][INFO]Scheduling job, id='HD_35e23ab9-4f03-48c1-825a-b15a503b4327_1' 
[2023-09-30T20:17:36.6794279Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_35e23ab9-4f03-48c1-825a-b15a503b4327_1' 
[2023-09-30T20:17:36.7147986Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_35e23a

{'runId': 'HD_35e23ab9-4f03-48c1-825a-b15a503b4327',
 'target': 'cluster-udacity',
 'status': 'Completed',
 'startTimeUtc': '2023-09-30T20:17:34.960482Z',
 'endTimeUtc': '2023-09-30T20:20:38.074708Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '9cd1cf8d-d9ae-45da-8bec-4ccba4e1e6f4',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': 'infinite_space_size',
  'score': '0.7896969696969697',
  'best_child_run_id': 'HD_35e23ab9-4f03-48c1-825a-b15a503b4327_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_35e23ab9-4f03-48c1-825a-b15a503b4327_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetry

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [98]:
hyperdrive_run.get_best_run_by_primary_metric()

Experiment,Id,Type,Status,Details Page,Docs Page
hyperdrive_experiment,HD_35e23ab9-4f03-48c1-825a-b15a503b4327_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [99]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
if best_run is None:
    raise Exception("No best run was found")

print(best_run)

best_run_params = best_run.get_details()['runDefinition']['parameters']

Run(Experiment: hyperdrive_experiment,
Id: HD_35e23ab9-4f03-48c1-825a-b15a503b4327_0,
Type: azureml.scriptrun,
Status: Completed)


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [108]:
#Register the model
description = 'HyperDrive best classification model for HR dataset'
model = best_run.register_model(model_name='HyperDriveModel', description=description, model_path='./outputs/HyperDrive_model.pkl')
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-242066', subscription_id='d7f39349-a66b-446e-aba6-0053c2cf1c11', resource_group='aml-quickstarts-242066'), name=HyperDriveModel, id=HyperDriveModel:1, version=1, tags={}, properties={})


We skip the rest of the cells since we are going to publish the model created by the AutoML process.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

